# Customer Churn Prediction

## Data Preprocesing

#### Loading Data

In [14]:
import pandas as pd
from datetime import datetime

In [15]:
raw_data_ = pd.read_csv('data/DS_CaseStudy_Churn_Pred_Data.csv')

In [16]:
raw_data = raw_data_.copy()

In [17]:
raw_data.head()

,delay,created_date,pickup_date,distance,driver_rating,fare,discount,payment_method,promocode,taximodelid,travel_status,trip_fare,profile_created,status_description,pre_booked,passengerid,tripid
0,NaN,20190809,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1,12,NaN,20170223,Unavailable,0,107206,7542464
1,8.38,20190701,20190701.0,16.49,NaN,676.13,0.0,1.0,NaN,1,1,676.13,20180201,Completed,0,123034,6044316
2,16.45,20190830,20190830.0,14.39,NaN,836.63,0.0,1.0,NaN,2,1,836.63,20180905,Completed,0,201136,1959290
3,NaN,20190709,NaN,NaN,NaN,NaN,NaN,1.0,lpp42,1,12,NaN,20180113,Unavailable,0,403293,1250879
4,NaN,20190718,NaN,NaN,NaN,NaN,NaN,1.0,abac,4,12,NaN,20190702,Unavailable,0,448530,884375


#### Date Columns preprocesing

In [18]:
raw_data['created_date'] = pd.to_datetime(raw_data['created_date'].astype(str))
raw_data['month'] = raw_data['created_date'].dt.month
raw_data['month'] = raw_data['month'].astype(str)

In [19]:
raw_data['profile_created'] = pd.to_datetime(raw_data['profile_created'].astype(str))

#### Seperating Data by months for labeling profiles

In [20]:
customer_data = raw_data[~(raw_data['month']=='8')] # June and July Data for generate customer profile(Behavioral)

In [21]:
label_refference_data = raw_data[(raw_data['month']=='8')] # August Data for labeling customer profiles

In [22]:
print(raw_data.shape)
print(customer_data.shape)
print(label_refference_data.shape)
print(raw_data['month'].unique())
print(customer_data['month'].unique())
print(label_refference_data['month'].unique())

(9004901, 18)
(6327474, 18)
(2677427, 18)
['8' '7' '6']
['7' '6']
['8']


In [24]:
set(customer_data['tripid'].unique()).intersection(set(label_refference_data['tripid'].unique()))


set()

In [25]:
customer_data.columns

Index(['delay', 'created_date', 'pickup_date', 'distance', 'driver_rating',
       'fare', 'discount', 'payment_method', 'promocode', 'taximodelid',
       'travel_status', 'trip_fare', 'profile_created', 'status_description',
       'pre_booked', 'passengerid', 'tripid', 'month'],
      dtype='object')

### Customer event data set Preposessing

#### Remove NAN rows by 'Fare' column

In [26]:
customer_data_filtered = customer_data.dropna(axis=0, subset=['fare'])

In [27]:
print(customer_data.shape)
print(customer_data_filtered.shape)
print(len(customer_data['passengerid'].unique()))
print(len(customer_data_filtered['passengerid'].unique()))

(6327474, 18)
(2694077, 18)
428961
384591


In [29]:
customer_data_filtered.isnull().sum(axis = 0)

delay                      10
created_date                0
pickup_date                10
distance                    0
driver_rating         2694077
fare                        0
discount                    0
payment_method              0
promocode             2081073
taximodelid                 0
travel_status               0
trip_fare                   0
profile_created             0
status_description          0
pre_booked                  0
passengerid                 0
tripid                      0
month                       0
dtype: int64

#### Fill null value of 'delay' by mean

In [31]:
customer_data_filtered['delay'] = customer_data_filtered['delay'].fillna(customer_data_filtered['delay'].mean())

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### Generate customer features  by grouping 'passengerid'  by aggrigating other selected columns

In [35]:
derive_features_dict = {'passengerid' : {'trip_total':'count'},
 'delay' : {'delay_total':'sum','delay_avg':'mean'},
 'distance' : {'distance_total':'sum','distance_avg':'mean'},
 'fare':{'fare_total':'sum','fare_avg':'mean'},
 'discount':{'discount_total':'sum','discount_avg':'mean'},
 'taximodelid':{'preffered_taximodelid':lambda x: x.mode()[0]},
 'trip_fare':{'trip_fare_total':'sum','trip_fare_avg':'mean'},
 'profile_created':{'profile_created':lambda x: x.mode()[0]},
 'payment_method':{'preffered_payment_method':lambda x: x.mode()[0]}                      
}

In [36]:
customer_data_agg = customer_data_filtered.groupby('passengerid').agg(derive_features_dict)
customer_data_agg = pd.DataFrame(customer_data_agg).reset_index(col_level=1)    
customer_data_agg.columns = customer_data_agg.columns.get_level_values(1)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


#### calculate customer time period (days) engaged  from profile created up to 2019/07/31 

In [37]:
customer_data_agg['profile_engaged_period'] = (pd.to_datetime('20190731') - customer_data_agg['profile_created']).dt.days

In [38]:
customer_data_agg.head()

,passengerid,trip_total,delay_total,delay_avg,distance_total,distance_avg,fare_total,fare_avg,discount_total,discount_avg,preffered_taximodelid,trip_fare_total,trip_fare_avg,profile_created,preffered_payment_method,profile_engaged_period
0,2,3,14.72,4.906667,16.41,5.470000,661.35,220.450000,0.0,0.0,1,661.35,220.450000,2018-01-02,1.0,575
1,4,2,17.63,8.815000,12.26,6.130000,506.36,253.180000,200.0,100.0,1,306.36,153.180000,2019-06-08,1.0,53
2,5,1,6.45,6.450000,12.44,12.440000,471.85,471.850000,0.0,0.0,1,471.85,471.850000,2019-01-09,1.0,203
3,7,52,486.82,9.361923,280.20,5.388462,12725.76,244.726154,0.0,0.0,1,12725.76,244.726154,2017-09-09,1.0,690
4,9,2,8.45,4.225000,6.80,3.400000,293.84,146.920000,0.0,0.0,1,293.84,146.920000,2017-12-19,1.0,589


#### preprocess 'pre_book' feature - Assign 3 categories and append to main dataframe

In [39]:
pre_booked = customer_data_filtered[['passengerid','pre_booked']]
pre_booked_unique = pre_booked.drop_duplicates(['passengerid','pre_booked'],keep= 'last')

In [40]:
pre_booked_unique= pre_booked_unique.sort_values(by=['passengerid','pre_booked'])
pre_booked_unique['pre_booked'] = pre_booked_unique['pre_booked'].astype(str)
pre_booked_joined = pre_booked_unique.groupby('passengerid')['pre_booked'].apply(','.join)
pre_booked_joined = pd.DataFrame(pre_booked_joined).reset_index(col_level=1)  


In [41]:
pre_booked_joined['pre_booked'].unique()

array(['0', '0,1', '1'], dtype=object)

In [45]:
merged_df = pd.merge(customer_data_agg, pre_booked_joined, on='passengerid')

In [46]:
merged_df.shape

(384591, 17)

In [47]:
merged_df.columns

Index(['passengerid', 'trip_total', 'delay_total', 'delay_avg',
       'distance_total', 'distance_avg', 'fare_total', 'fare_avg',
       'discount_total', 'discount_avg', 'preffered_taximodelid',
       'trip_fare_total', 'trip_fare_avg', 'profile_created',
       'preffered_payment_method', 'profile_engaged_period', 'pre_booked'],
      dtype='object')

In [48]:
merged_df.head()

,passengerid,trip_total,delay_total,delay_avg,distance_total,distance_avg,fare_total,fare_avg,discount_total,discount_avg,preffered_taximodelid,trip_fare_total,trip_fare_avg,profile_created,preffered_payment_method,profile_engaged_period,pre_booked
0,2,3,14.72,4.906667,16.41,5.470000,661.35,220.450000,0.0,0.0,1,661.35,220.450000,2018-01-02,1.0,575,0
1,4,2,17.63,8.815000,12.26,6.130000,506.36,253.180000,200.0,100.0,1,306.36,153.180000,2019-06-08,1.0,53,0
2,5,1,6.45,6.450000,12.44,12.440000,471.85,471.850000,0.0,0.0,1,471.85,471.850000,2019-01-09,1.0,203,0
3,7,52,486.82,9.361923,280.20,5.388462,12725.76,244.726154,0.0,0.0,1,12725.76,244.726154,2017-09-09,1.0,690,0
4,9,2,8.45,4.225000,6.80,3.400000,293.84,146.920000,0.0,0.0,1,293.84,146.920000,2017-12-19,1.0,589,0


### labeling customer data by using August data set

In [50]:
label_unque_df = label_refference_data[['passengerid']].drop_duplicates(['passengerid'],keep= 'last').reset_index(drop = True)

In [52]:
label_unque_df['churn'] = 'NO'

In [53]:
customer_prediction_preprocessed_data = merged_df.merge(label_unque_df, how = 'left', on = 'passengerid')

In [54]:
customer_prediction_preprocessed_data['churn'] = customer_prediction_preprocessed_data['churn'].fillna('YES')

In [55]:
customer_prediction_preprocessed_data['churn'].value_counts()

NO     234390
YES    150201
Name: churn, dtype: int64

In [56]:
len(set(merged_df['passengerid'].unique()).intersection(set(label_unque['passengerid'].unique())))

234390

In [57]:
customer_prediction_preprocessed_data.to_csv('data/customer_preprocessed_data.csv', index=False)

In [58]:
customer_prediction_preprocessed_data.columns

Index(['passengerid', 'trip_total', 'delay_total', 'delay_avg',
       'distance_total', 'distance_avg', 'fare_total', 'fare_avg',
       'discount_total', 'discount_avg', 'preffered_taximodelid',
       'trip_fare_total', 'trip_fare_avg', 'profile_created',
       'preffered_payment_method', 'profile_engaged_period', 'pre_booked',
       'churn'],
      dtype='object')